# Feature selection with Linear Regression

The second phase of this project is going to reduce the scope of our data by looking for specific conditions that are strongly coorelated with both of our predictors (snap and unemployment).
To do this we will run linear regression on the data and examine the correlation coefficients.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Linear Regression") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/22 19:55:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
#spark.stop()

In [13]:
dataset_path = 'hdfs://columbia.cs.colostate.edu:31175/final/uber/services_and_health.parquet'

full_set = spark.read.parquet(dataset_path)

full_set.show()

+-----+----+----------------------------------------------------------+---------------------------------------------------------------+----------------------+--------------------------------+--------------------------------------------------+-------------------------------------------+-------------------------------------------------+------------------------------------------------+------------------------------------------------------+---------------------------------------+--------------------------------------+--------------------------------------------+-----------------------------------------------------------------+-----------------------------------------+----------------------------------------------+--------------------------------------------------------------------------+-------------------------------------------------------+------------------------------------+---------------------------------------------------------+-------------------------------------------------------

Here we clean the data (removing special chars from the column titles that caused issues) and do the linear regression.

In [21]:
from pyspark.sql import DataFrame

def clean_column_names(df: DataFrame) -> DataFrame:
    for col_name in df.columns:
        new_col_name = col_name.replace(".", "").replace(" - ", "_").replace(" ", "_")
        df = df.withColumnRenamed(col_name, new_col_name)
    return df

data_cleaned = clean_column_names(full_set)

print(data_cleaned.columns)


['state', 'year', '064_INTRACRANIAL_HEMORRHAGE_OR_CEREBRAL_INFARCTION_W_MCC', '066_INTRACRANIAL_HEMORRHAGE_OR_CEREBRAL_INFARCTION_W/O_CC/MCC', '101_SEIZURES_W/O_MCC', '176_PULMONARY_EMBOLISM_W/O_MCC', '177_RESPIRATORY_INFECTIONS_&_INFLAMMATIONS_W_MCC', '189_PULMONARY_EDEMA_&_RESPIRATORY_FAILURE', '190_CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_W_MCC', '191_CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_W_CC', '192_CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_W/O_CC/MCC', '193_SIMPLE_PNEUMONIA_&_PLEURISY_W_MCC', '194_SIMPLE_PNEUMONIA_&_PLEURISY_W_CC', '195_SIMPLE_PNEUMONIA_&_PLEURISY_W/O_CC/MCC', '208_RESPIRATORY_SYSTEM_DIAGNOSIS_W_VENTILATOR_SUPPORT_<96_HOURS', '238_MAJOR_CARDIOVASC_PROCEDURES_W/O_MCC', '243_PERMANENT_CARDIAC_PACEMAKER_IMPLANT_W_CC', '246_PERC_CARDIOVASC_PROC_W_DRUG-ELUTING_STENT_W_MCC_OR_4+_VESSELS/STENTS', '247_PERC_CARDIOVASC_PROC_W_DRUG-ELUTING_STENT_W/O_MCC', '253_OTHER_VASCULAR_PROCEDURES_W_CC', '280_ACUTE_MYOCARDIAL_INFARCTION,_DISCHARGED_ALIVE_W_MCC', '281_ACUTE_MYOCARDIAL_INFARCTI

In [22]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import col

def is_feature(col):
    decision = True
    decision = decision and not col.startswith('avg')
    decision = decision and not col.startswith('state')
    decision = decision and not col.startswith('year')
    return decision

drg_columns = [col for col in data_cleaned.columns if is_feature(col)] 

print(drg_columns)

assembler = VectorAssembler(inputCols=drg_columns, outputCol="features")
df_features = assembler.transform(data_cleaned)

data_snap = df_features.select("features", col("avg_snap").alias("label"))
data_unemployment = df_features.select("features", col("avg_unemployment").alias("label"))

['064_INTRACRANIAL_HEMORRHAGE_OR_CEREBRAL_INFARCTION_W_MCC', '066_INTRACRANIAL_HEMORRHAGE_OR_CEREBRAL_INFARCTION_W/O_CC/MCC', '101_SEIZURES_W/O_MCC', '176_PULMONARY_EMBOLISM_W/O_MCC', '177_RESPIRATORY_INFECTIONS_&_INFLAMMATIONS_W_MCC', '189_PULMONARY_EDEMA_&_RESPIRATORY_FAILURE', '190_CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_W_MCC', '191_CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_W_CC', '192_CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_W/O_CC/MCC', '193_SIMPLE_PNEUMONIA_&_PLEURISY_W_MCC', '194_SIMPLE_PNEUMONIA_&_PLEURISY_W_CC', '195_SIMPLE_PNEUMONIA_&_PLEURISY_W/O_CC/MCC', '208_RESPIRATORY_SYSTEM_DIAGNOSIS_W_VENTILATOR_SUPPORT_<96_HOURS', '238_MAJOR_CARDIOVASC_PROCEDURES_W/O_MCC', '243_PERMANENT_CARDIAC_PACEMAKER_IMPLANT_W_CC', '246_PERC_CARDIOVASC_PROC_W_DRUG-ELUTING_STENT_W_MCC_OR_4+_VESSELS/STENTS', '247_PERC_CARDIOVASC_PROC_W_DRUG-ELUTING_STENT_W/O_MCC', '253_OTHER_VASCULAR_PROCEDURES_W_CC', '280_ACUTE_MYOCARDIAL_INFARCTION,_DISCHARGED_ALIVE_W_MCC', '281_ACUTE_MYOCARDIAL_INFARCTION,_DISCHARGED_AL

In [23]:
data_snap.show()

+--------------------+--------------------+
|            features|               label|
+--------------------+--------------------+
|[12366.0,7046.0,6...|   9151211.733333332|
|[10976.0,4532.0,4...|        4.65712415E7|
|[12022.0,5597.0,4...|  2879850.1428571427|
|[10069.0,4557.0,5...|  2506513.2608695654|
|[51847.0,5256.0,7...|           1623640.2|
|[14731.0,7396.0,7...|         1.5817335E7|
|[10230.0,4778.0,4...|    6857740.71969697|
|[10586.0,4826.0,4...|   901454.6390977444|
|[11560.0,5527.0,4...|  491554.93434343435|
|[12886.0,5303.0,4...|  2187851.6101694917|
|[14428.0,7140.0,7...|  243215.50218340612|
|[20811.0,7937.0,8...|1.0943420965517242E7|
|[12214.0,5301.0,4...|   1615027.889937107|
|[12432.0,7558.0,6...|  3878353.1041666665|
|[13762.0,6735.0,7...|       1.397572075E8|
|[12563.0,5186.0,5...|  139581.42156862744|
|[16836.0,7114.0,6...|           6187163.0|
|[15943.0,6862.0,6...|   275438.0616740088|
|[13139.0,5679.0,6...|        1.05110731E8|
|[11697.0,5119.0,5...|         1

In [30]:
def run_regression(data, label):
    lr = LinearRegression(featuresCol='features', labelCol='label')
    model = lr.fit(data)

    coefs = model.coefficients
    print("Coefficients for " + label + ": " + str(coefs))
    print("Intercept for "+ label + ": " + str(model.intercept))
    return coefs
    

snap_coefs = run_regression(data_snap, 'SNAP')
unemployment_coefs = run_regression(data_unemployment, 'Unemployment')


24/04/22 20:25:15 WARN Instrumentation: [8da5d4f4] regParam is zero, which might cause numerical instability and overfitting.


Coefficients for SNAP: [1275.035719882323,-21.096334919689212,4692.976524647587,11121.26591920684,-3497.9587949410375,-1713.2564184401688,-11655.306676829603,-7114.723559536132,15293.315119458544,3606.766366347151,7128.40279700787,-9220.822918091793,-2676.1287959119095,-2541.337692562382,-968.4752857673469,4025.3085642634046,282.79048179560806,2542.256149286368,-104.20396005051684,987.8070111302094,-4811.984694122666,2031.5845450733682,35667.53069262808,14514.887534754433,15508.745525033548,-4503.42158949936,-12861.014743315296,5916.282923346511,16923.473777839412,-2805.9783549095414,-512.4614009364301,-874.7952255894868,-1815.3847358208084,2542.531544663945,-21115.030806073686,-11922.76180923528,-819.8785059733291,-1987.1221479106878,-2483.826509742737,-116.12092674491893,11942.709322201765,5617.800125729247,-21191.857764705524,-8211.979121757355,-6432.819698592843,-1351.6902798353606,-779.8486762916239,-1462.862987716325,-7940.953773226935,-3798.0301702624024,540.8731986746814,10693.

24/04/22 20:25:16 WARN Instrumentation: [d441b20b] regParam is zero, which might cause numerical instability and overfitting.


Coefficients for Unemployment: [-0.00010636422874742619,-0.00020968475353763932,-0.0002190495340254594,-5.3632506594367723e-05,0.00023734361341917908,3.263703442935933e-05,7.410115221423473e-05,0.00025244198969079406,0.0005976139591922438,0.00011232898704639687,-0.00026931907183537143,0.00044460682707501026,-0.00010645541953818805,3.0949474179318265e-05,-9.332247820169166e-06,4.2402975545317045e-05,-3.792878067325902e-06,1.5709508235205543e-05,-1.0216628045552844e-05,-6.364244332418433e-05,0.0002906657220813679,-0.0002866918805563547,-0.00019468236463155873,0.000560190863096428,-1.5002106276186073e-05,-0.0005484352863637039,0.00037071541709512144,0.00036801393492445533,-0.0011093854821399153,5.102569642117739e-05,4.9925594691091483e-05,3.097558021294957e-05,0.00011284519683332715,-0.0004790433908611836,-0.0002211991538420696,-0.00019158201106134185,9.043239939766811e-05,-2.349487886097693e-05,3.96096137576633e-05,-9.899120108812761e-05,-0.0005851261382278435,0.0006704809424038297,-0.00

Here we can examine the conditions with the highest correlations and make our decision on which fields to analyize in the next step.

In [34]:
from pyspark.sql import Row

def get_coeffs_table(coefs):
    drg_and_coefs = [(drg, float(coef)) for drg, coef in zip(drg_columns, coefs)]
    df_coefs = spark.createDataFrame(drg_and_coefs, ["DRG", "Coefficient"]).orderBy(col("Coefficient").desc())
    return df_coefs

df_coefs_snap = get_coeffs_table(snap_coefs)
df_coefs_snap.show()

+--------------------+------------------+
|                 DRG|       Coefficient|
+--------------------+------------------+
|292_HEART_FAILURE...| 35667.53069262808|
|      313_CHEST_PAIN|16923.473777839412|
|308_CARDIAC_ARRHY...|15508.745525033548|
|192_CHRONIC_OBSTR...|15293.315119458544|
|293_HEART_FAILURE...|14514.887534754433|
|481_HIP_&_FEMUR_P...|11942.709322201765|
|176_PULMONARY_EMB...| 11121.26591920684|
|871_SEPTICEMIA_OR...| 10693.20220436549|
|194_SIMPLE_PNEUMO...|  7128.40279700787|
|948_SIGNS_&_SYMPT...|6753.3237046528275|
|312_SYNCOPE_&_COL...| 5916.282923346511|
|552_MEDICAL_BACK_...| 5617.800125729247|
|101_SEIZURES_W/O_MCC| 4692.976524647587|
|246_PERC_CARDIOVA...|4025.3085642634046|
|193_SIMPLE_PNEUMO...| 3606.766366347151|
|378_GI_HEMORRHAGE...| 2542.531544663945|
|253_OTHER_VASCULA...| 2542.256149286368|
|291_HEART_FAILURE...|2031.5845450733682|
|064_INTRACRANIAL_...| 1275.035719882323|
|281_ACUTE_MYOCARD...| 987.8070111302094|
+--------------------+------------

In [35]:
df_coefs_snap.orderBy(col("Coefficient").asc()).show()

+--------------------+-------------------+
|                 DRG|        Coefficient|
+--------------------+-------------------+
|603_CELLULITIS_W/...|-21191.857764705524|
|389_GI_OBSTRUCTIO...|-21115.030806073686|
|310_CARDIAC_ARRHY...|-12861.014743315296|
|392_ESOPHAGITIS,_...| -11922.76180923528|
|190_CHRONIC_OBSTR...|-11655.306676829603|
|195_SIMPLE_PNEUMO...| -9220.822918091793|
|   638_DIABETES_W_CC| -8211.979121757355|
|690_KIDNEY_&_URIN...| -7940.953773226935|
|191_CHRONIC_OBSTR...| -7114.723559536132|
|640_MISC_DISORDER...| -6432.819698592843|
|287_CIRCULATORY_D...| -4811.984694122666|
|309_CARDIAC_ARRHY...|  -4503.42158949936|
|812_RED_BLOOD_CEL...|-3798.0301702624024|
|177_RESPIRATORY_I...|-3497.9587949410375|
|314_OTHER_CIRCULA...|-2805.9783549095414|
|897_ALCOHOL/DRUG_...|-2713.0290208662914|
|208_RESPIRATORY_S...|-2676.1287959119095|
|238_MAJOR_CARDIOV...| -2541.337692562382|
|470_MAJOR_JOINT_R...| -2483.826509742737|
|460_SPINAL_FUSION...|-1987.1221479106878|
+----------

In [36]:
df_coefs_unemployment = get_coeffs_table(unemployment_coefs)
df_coefs_unemployment.show()
df_coefs_unemployment.orderBy(col("Coefficient").asc()).show()

+--------------------+--------------------+
|                 DRG|         Coefficient|
+--------------------+--------------------+
|552_MEDICAL_BACK_...|6.704809424038297E-4|
|872_SEPTICEMIA_OR...|6.350460871716643E-4|
|192_CHRONIC_OBSTR...|5.976139591922438E-4|
|293_HEART_FAILURE...| 5.60190863096428E-4|
|195_SIMPLE_PNEUMO...|4.446068270750102...|
|641_MISC_DISORDER...|3.829447294042901E-4|
|310_CARDIAC_ARRHY...|3.707154170951214...|
|312_SYNCOPE_&_COL...|3.680139349244553...|
|690_KIDNEY_&_URIN...|3.307931572722216E-4|
|287_CIRCULATORY_D...|2.906657220813679E-4|
|191_CHRONIC_OBSTR...|2.524419896907940...|
|177_RESPIRATORY_I...|2.373436134191790...|
|871_SEPTICEMIA_OR...|2.338356661577094...|
|812_RED_BLOOD_CEL...|1.520993155133804...|
|640_MISC_DISORDER...|1.224054314227436...|
|377_GI_HEMORRHAGE...|1.128451968333271...|
|193_SIMPLE_PNEUMO...|1.123289870463968...|
|394_OTHER_DIGESTI...|9.043239939766811E-5|
|190_CHRONIC_OBSTR...|7.410115221423473E-5|
|682_RENAL_FAILURE...|6.52652138

By comparing the most correlated drg fields between snap enrollment and unemployment, we have curated a short list of predictors that are strongly correlated to both:

* 292 - Heart Failure
* 313 - Chest Pain
* 603 - Cellulitus
* 192 - COPD
* 195 - Simple Pneumonia
* 871 - Septicemia w/ MCC

Next we will reduce the dataset to just include these drg fields

In [37]:
shortlist_prefixes = ['292','313','603','192','195','871']

def keep_col(column):
    def unless(decision, prefix):
        return decision or column.startswith(prefix)
    
    decision = False
    decision = unless(decision, 'state')
    decision = unless(decision, 'year')
    decision = unless(decision, 'avg') # both snap and unemployment
    
    for prefix in shortlist_prefixes:
        decision = unless(decision, prefix)

    return decision

columns_to_keep = [col for col in data_cleaned.columns if keep_col(col)]

focused_data = data_cleaned.select(columns_to_keep)

focused_data.show()

+-----+----+----------------------------------------------------+------------------------------------------+------------------------------+--------------+----------------------+------------------------------------------------------+--------------------+------------------+
|state|year|192_CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_W/O_CC/MCC|195_SIMPLE_PNEUMONIA_&_PLEURISY_W/O_CC/MCC|292_HEART_FAILURE_&_SHOCK_W_CC|313_CHEST_PAIN|603_CELLULITIS_W/O_MCC|871_SEPTICEMIA_OR_SEVERE_SEPSIS_W/O_MV_96+_HOURS_W_MCC|            avg_snap|  avg_unemployment|
+-----+----+----------------------------------------------------+------------------------------------------+------------------------------+--------------+----------------------+------------------------------------------------------+--------------------+------------------+
|   AZ|2013|                                                5393|                                      6584|                          6790|          3539|                  5678|    

In [39]:
output_path_parquet = 'hdfs://columbia.cs.colostate.edu:31175/final/focused/focused_data.parquet'
focused_data.write.mode('overwrite').parquet(output_path_parquet)

output_path_csv = 'hdfs://columbia.cs.colostate.edu:31175/final/focused/focused_data.csv'
focused_data.write.option("header", "true").mode('overwrite').csv(output_path_csv)

Before finishing this session and moving on the the machine learning analysis, we will extract information about the normal range of snap and unemployment. These will be useful in moderating the predictions of our models, so that they don't produce unrealistic or nonesensical results.

In [38]:
from pyspark.sql.functions import min, max

min_max_snap = data_cleaned.agg(
    min("avg_snap").alias("min_snap"),
    max("avg_snap").alias("max_snap")
)

min_max_unemployment = data_cleaned.agg(
    min("avg_unemployment").alias("min_unemployment"),
    max("avg_unemployment").alias("max_unemployment")
)

print("Minimum and Maximum SNAP Enrollment:")
min_max_snap.show()

print("Minimum and Maximum Unemployment Rates:")
min_max_unemployment.show()


Minimum and Maximum SNAP Enrollment:
+------------------+-------------+
|          min_snap|     max_snap|
+------------------+-------------+
|122834.27450980392|4.602632065E8|
+------------------+-------------+

Minimum and Maximum Unemployment Rates:
+-----------------+------------------+
| min_unemployment|  max_unemployment|
+-----------------+------------------+
|2.616666666666667|11.608333333333334|
+-----------------+------------------+

